### Importações e Configurações

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Caminho para Salvar os Dados Limpos

path_data = './outputs/data/'

### Carregando os Dados

In [4]:
# Train.csv

df_train_raw = pd.read_csv('../../data/raw/train.csv', delimiter=',')
df_train_raw.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [5]:
# Stores.csv

df_stores_raw = pd.read_csv('../../data/raw/stores.csv', delimiter=',')
df_stores_raw.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [6]:
# Features.csv

df_features_raw = pd.read_csv('../../data/raw/features.csv', delimiter=',')
df_features_raw.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


Os dados são carregados diretamente da pasta raw, que representa a versão original e imutável do dataset. A partir deste notebook, qualquer modificação será aplicada apenas em versões processadas.

In [7]:
def convert_date(df):
    date_cols = []
    for col in df.columns:
        if col.lower().startswith('date') and df[col].notna().any():
            converted = pd.to_datetime(df[col], errors='coerce')

            if converted.notna().sum() >= df[col].notna().sum():
                df[col] = converted
                date_cols.append(col)
    
    print('Colunas Convertidas')
    for col in date_cols:
        n_nat = df[col].isna().sum()
        print(f'{col} convertida para datetime64, quantidades de valores "NaT" encontrados: {n_nat}')
    
    return df

In [8]:
df_train = convert_date(df_train_raw)

Colunas Convertidas
Date convertida para datetime64, quantidades de valores "NaT" encontrados: 0


In [9]:
df_features = convert_date(df_features_raw)

Colunas Convertidas
Date convertida para datetime64, quantidades de valores "NaT" encontrados: 0


A conversão explícita da coluna de datas garante consistência nas operações de merge e possibilita futuras transformações temporais, além de evitar erros silenciosos durante o processamento.

### Merge dos Datasets

In [10]:
df = (
    df_train
        .merge(df_features, on=['Store', 'Date'], how='left')
        .merge(df_stores_raw, on='Store', how='left')
)

O dataset final é construído a partir da junção das vendas com as variáveis externas e informações das lojas. O uso de left join preserva todas as observações de vendas, evitando perda de informações do target.

### Definição da Variável Target

In [11]:
TARGET = 'Weekly_Sales'

A variável Weekly_Sales é definida explicitamente como target. Todas as decisões de limpeza e transformação a partir deste ponto são avaliadas com base no impacto potencial sobre a regressão linear.

### Remoção das Vendas Negativas

In [12]:
df = df[df[TARGET] >= 0].copy()

Valores negativos de vendas não possuem interpretação econômica direta neste contexto. Essas observações são removidas para evitar distorções na estimação dos coeficientes dos modelos.

### Tratamento de Valores Ausentes

In [13]:
# Colunas MarkDown

markdown_cols = [col for col in df.columns if 'MarkDown' in col]
df[markdown_cols] = df[markdown_cols].fillna(0)

Os valores ausentes nas variáveis de MarkDown são interpretados como ausência de ações promocionais. Por isso, a imputação com zero preserva o significado econômico dessas variáveis.

In [14]:
df.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday_x', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y', 'Type', 'Size'],
      dtype='object')

In [15]:
# Colunas CPI e Unemployment

df['CPI'] = df.groupby('Store')['CPI'].transform(lambda x: x.fillna(x.median()))
df['Unemployment'] = df.groupby('Store')['Unemployment'].transform(lambda x: x.fillna(x.median()))

CPI e taxa de desemprego variam de forma gradual ao longo do tempo. A imputação pela mediana por loja preserva o padrão local e evita a exclusão desnecessária de observações.

### Transformação do Target

In [16]:
df['Weekly_Sales_log'] = np.log1p(df['Weekly_Sales'])

Em vez de remover observações extremas, opta-se pela transformação logarítmica da variável alvo, reduzindo a assimetria e tornando os dados mais adequados à regressão linear.

### Verificação Final e Salvamento

In [17]:
# Verificação

df.isna().sum().sort_values(ascending=False)

Store               0
Dept                0
Date                0
Weekly_Sales        0
IsHoliday_x         0
Temperature         0
Fuel_Price          0
MarkDown1           0
MarkDown2           0
MarkDown3           0
MarkDown4           0
MarkDown5           0
CPI                 0
Unemployment        0
IsHoliday_y         0
Type                0
Size                0
Weekly_Sales_log    0
dtype: int64

Esta verificação final garante que as principais variáveis estejam livres de valores ausentes antes de avançarmos.

In [18]:
# Salvamento do Dados Limpos

df.to_csv(f'{path_data}df_clean.csv', index=False)

O dataset limpo é salvo separadamente para garantir reprodutibilidade e isolamento das decisões de limpeza. A partir deste ponto, todas as análises subsequentes utilizam esta versão processada.